In [1]:
import pandas as pd
import numpy as np
import pickle
from igraph import *
from tqdm.notebook import tqdm
import matplotlib
%matplotlib inline



In [2]:
df = pd.read_pickle('../../data/dataset.p')

### 1.Subject areas

In [49]:
journals = df.filter(['publicationName','subject_areas'])

journals['subject_areas'] = journals.apply(lambda x: [l[0] for l in x.subject_areas],axis=1)

journals[['subject_areas_0','subject_areas_1','subject_areas_2','subject_areas_3','subject_areas_4','subject_areas_5']] = pd.DataFrame(journals.subject_areas.tolist())

journals.subject_areas = None

## subfields

In [53]:
quant = ['scientometrics','journal of informetrics']
hist_phil = ['british journal for the history of science','isis','science and education',            
             'science and technology studies','science, technology and society','social studies of science',
             'studies in history and philosophy of science','synthese']
management = ['research policy','science and public policy']
social_sci = ['minerva','public understanding of science','research evaluation', 'science, technology and human values']

df.loc[df.publicationName.isin(quant), 'field'] = 'Library and Information Sciences'
df.loc[df.publicationName.isin(management), 'field'] = 'Management'
df.loc[df.publicationName.isin(social_sci), 'field'] = 'Education, Communication and Anthropology'
df.loc[df.publicationName.isin(hist_phil), 'field'] = 'History and Philosophy'
#df.loc[df.publicationName.isin(education), 'field'] = 'Education'



In [55]:
def most_cited(series):

    return set(df.title[df.citedby_count == max(series)].values)

In [56]:
def mean_round(x):
    return round(np.mean(x),2)

In [67]:
def make_summary(df):

    summary_stats = df.groupby(['publicationName','field']).citedby_count.agg(['count', mean_round, 'max']).sort_values(by =['field','mean_round'])
    summary_stats.reset_index(inplace=True)

    summary_stats['publicationName'] = summary_stats.apply(lambda x: x.publicationName.title(),axis=1)
    total = ['Total',' ',summary_stats['count'].sum(),mean_round(summary_stats['mean_round']),summary_stats['max'].max()]

    summary_stats.loc['Total']= total

    summary_stats = summary_stats.sort_values(by=['field','count'], ascending=False)

    summary_stats.set_index(['field', 'publicationName'],inplace=True)
    return summary_stats

In [69]:
summary_stats = make_summary(df)

In [70]:
print(summary_stats.to_latex(index=True))

\begin{tabular}{llrrr}
\toprule
  &       &  count &  mean\_round &   max \\
field & publicationName &        &             &       \\
\midrule
Management & Research Policy &   3221 &       83.75 &  4820 \\
  & Science And Public Policy &   1707 &       13.27 &   462 \\
Library and Information Sciences & Scientometrics &   5136 &       20.04 &  1334 \\
  & Journal Of Informetrics &    876 &       22.63 &   352 \\
History and Philosophy & Synthese &   4151 &        8.53 &   910 \\
  & Social Studies Of Science &   1069 &       40.95 &  4709 \\
  & Science And Education &   1034 &       11.60 &   298 \\
  & Studies In History And Philosophy Of Science &    911 &        8.76 &   145 \\
  & Isis &    523 &       12.47 &   415 \\
  & Science, Technology And Society &    345 &        6.07 &   122 \\
  & British Journal For The History Of Science &    276 &        9.57 &    88 \\
  & Science And Technology Studies &    111 &        5.29 &    39 \\
Education, Communication and Anthropology & P

# 2.Network

In [12]:
raw_dir = '../../data/torch/bert/'
names = ['x.p', 'edge_pairs.p', 'eid_row_indices.p']
x,edge_pairs,eid_row_indices = [pickle.load(open('{}/{}'.format(raw_dir, name), 'rb'))for name in names]



In [13]:
edge_pairs = edge_pairs.detach().numpy()

edge_pairs_df = pd.DataFrame(edge_pairs).T

_ = edge_pairs_df.merge(eid_row_indices, left_on=0, right_on='index')
_ = _.merge(eid_row_indices, left_on=1, right_on='index')

edges = _.loc[:,['eid_x','eid_y']]

In [14]:
g = Graph.TupleList(edges.values,directed=False)

descriptive statistics

In [90]:
g.average_path_length()

6.1414321323727865

In [91]:
np.mean(g.degree())

8.29979490891543

In [92]:
np.max(g.degree())

282

In [93]:
g.vcount()

16578

In [94]:
len(set(edges.eid_x).union(set(edges.eid_y)))

16578

In [95]:
g.ecount()

68797

In [98]:
g.assortativity_degree()

-0.0060198543679408365

In [99]:
np.mean(g.betweenness())

37807.8781517674

In [104]:
g.is_connected()

False

In [100]:
comps= g.components()

In [101]:
giant = comps.giant()

In [102]:
giant.ecount()

68168

In [105]:
giant.vcount()

15615

In [109]:
print('Edges not in the giant component: {}'.format(g.ecount()-giant.ecount()))
print('Vertices not in the giant component: {}'.format(g.vcount()-giant.vcount()))

Edges not in the giant component: 629
Vertices not in the giant component: 963


In [112]:
np.mean(giant.closeness())

0.17105498895935597

In [114]:
np.mean(giant.eigenvector_centrality())
np.mean(g.eigenvector_centrality())

0.011214141865822723

In [115]:
g.farthest_points()

(15475, 15550, 37)

In [116]:
g.diameter()

37

In [120]:
g.summary()

'IGRAPH UN-- 16578 68797 -- \n+ attr: name (v)'

### small word

In [156]:
#clustering coefficient
clust_coef = g.transitivity_undirected()
clust_coef

0.08137529739399864

In [157]:
average_path_length = g.average_path_length()
average_path_length

6.1414321323727865

### random model

In [143]:
def random_model(n = g.vcount(), m = g.ecount(), times=100):
    rnd_clust_coefs = []
    rnd_lengths = []
    for i in tqdm(range(times)):
        rnd_graph = Graph.Erdos_Renyi(n=n,m=m)
        clust = rnd_graph.transitivity_undirected()
        length = rnd_graph.average_path_length()
        rnd_clust_coefs += [clust]
        rnd_lengths += [length]
    return rnd_clust_coefs, rnd_lengths

In [149]:
rnd_clust_coefs, rnd_lengths = random_model()

In [153]:
mean_random_clust_coef = np.mean(rnd_clust_coefs)
mean_random_clust_coef

0.0005009197590637582

In [158]:
mean_random_length = np.mean(rnd_lengths)
mean_random_length

4.8269054722968345

In [159]:
clust_coef/mean_random_clust_coef

162.45176182726905

In [160]:
average_path_length / mean_random_length

1.2723332096765607

In [ ]:
average_path_length

### 3. Differences between dataset and network 

In [78]:
network_ids = [line.rstrip('\n') for line in open('../../results/network_ids.txt')]

In [79]:
in_net_df = df[df.eid.isin(network_ids)]
out_net_df = df[-df.eid.isin(network_ids)]

In [81]:
out_net_df.citedby_count.mean()

18.99228422752557

In [82]:
out_net_df.loc[:,'coverDate'] = out_net_df.loc[:,'coverDate'].apply(lambda x: int(x.rsplit('-')[0]))

out_net_df.coverDate.mean()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


2001.289790059214

#### Out of network stats

In [75]:
oon_summary_stats = make_summary(out_net_df)
oon_summary_stats

count  \
field                                     publicationName                                       
Management                                Research Policy                                 643   
                                          Science And Public Policy                       637   
Library and Information Sciences          Scientometrics                                  784   
                                          Journal Of Informetrics                          12   
History and Philosophy                    Synthese                                       1702   
                                          Studies In History And Philosophy Of Science    372   
                                          Isis                                            250   
                                          Science And Education                           250   
                                          British Journal For The History Of Science      145   
                                          Social Studies Of Science                       139   
                                          Science, Technology And Society                 134   
                                          Science And Technology Studies                    8   
Education, Communication and Anthropology Public Understanding Of Science                 170   
                                          Research Evaluation                             142   
                                          Science, Technology And Human Values            106   
                                          Minerva                                          79   
                                          Total                                          5573   

                                                                                        mean_round  \
field                                     publicationName                                            
Management                                Research Policy                                    79.72   
                                          Science And Public Policy                          10.31   
Library and Information Sciences          Scientometrics                                     19.73   
                                          Journal Of Informetrics                            13.50   
History and Philosophy                    Synthese                                            6.70   
                                          Studies In History And Philosophy Of Science        7.68   
                                          Isis                                                9.09   
                                          Science And Education                              10.92   
                                          British Journal For The History Of Science          9.03   
                                          Social Studies Of Science                          24.71   
                                          Science, Technology And Society                     4.15   
                                          Science And Technology Studies                      2.25   
Education, Communication and Anthropology Public Understanding Of Science                    25.32   
                                          Research Evaluation                                 6.15   
                                          Science, Technology And Human Values               19.51   
                                          Minerva                                             7.24   
                                          Total                                              16.00   

                                                                                         max  
field                                     publicationName                                     
Management                                Research Policy                               3404  
                    

#### In network stats

In [85]:
in_net_summary = make_summary(in_net_df)
in_net_summary

count  \
field                                     publicationName                                       
Management                                Research Policy                                2578   
                                          Science And Public Policy                      1070   
Library and Information Sciences          Scientometrics                                 4352   
                                          Journal Of Informetrics                         864   
History and Philosophy                    Synthese                                       2449   
                                          Social Studies Of Science                       930   
                                          Science And Education                           784   
                                          Studies In History And Philosophy Of Science    539   
                                          Isis                                            273   
                                          Science, Technology And Society                 211   
                                          British Journal For The History Of Science      131   
                                          Science And Technology Studies                  103   
Education, Communication and Anthropology Public Understanding Of Science                 807   
                                          Science, Technology And Human Values            651   
                                          Research Evaluation                             524   
                                          Minerva                                         312   
                                          Total                                         16578   

                                                                                        mean_round  \
field                                     publicationName                                            
Management                                Research Policy                                    84.75   
                                          Science And Public Policy                          15.03   
Library and Information Sciences          Scientometrics                                     20.10   
                                          Journal Of Informetrics                            22.76   
History and Philosophy                    Synthese                                            9.80   
                                          Social Studies Of Science                          43.38   
                                          Science And Education                              11.82   
                                          Studies In History And Philosophy Of Science        9.50   
                                          Isis                                               15.57   
                                          Science, Technology And Society                     7.28   
                                          British Journal For The History Of Science         10.16   
                                          Science And Technology Studies                      5.52   
Education, Communication and Anthropology Public Understanding Of Science                    26.04   
                                          Science, Technology And Human Values               35.04   
                                          Research Evaluation                                15.04   
                                          Minerva                                            18.86   
                                          Total                                              21.92   

                                                                                         max  
field                                     publicationName                                     
Management                                Research Policy                               4820  
                    

# Topic Modelling distribution

In [ ]:
LDA = LDA_wrapper()

In [ ]:
lda_model = LDA.restore_model("../../results/lda_model20t.p")
texts = df['text_clean'].values
data_vectorized, vectorizer = LDA.data_vectorizer(data=texts)

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [ ]:
print_topics(lda_model, vectorizer, 5)


selected topics: 
The LDAViz change the topics order, so we need to map them. 


|LDAViz | lda_model|
|---|--|
|1 | 18|
|2 | 6|
|3 | 16|
|4 | 11|
|5 | 19|
|6 | 7|
|7 | 16|
|8 | 9|
|9 | 8|
|10 | 14|
|11 | 10|
|12 | 12|
|13 | 5|
|14 | 13|
|15 | 2|
|16 | 4|
|17 | 3|
|18 | 0|
|19 | 15|
|20 | 1|



In [ ]:
def get_topics(x):
    topic_1 = x[18]
    topic_2 = x[6]
    topic_3 = x[16]
    topic_4 = x[11]
    topic_5 = x[19]
    topic_6 = x[7]
    topic_7 = x[16]
    topic_8 = x[9]
    topic_9 = x[8]
    topic_10 = x[14]
    topic_11 = x[10]
    topic_12 = x[12]
    topic_13 = x[5]
    topic_14 = x[13]
    topic_15 = x[2]
    topic_16 = x[4]
    topic_17 = x[3]
    topic_18 = x[0]
    topic_19 = x[15]
    topic_20 = x[1]


    return{'topic_1':topic_1,
    'topic_2':topic_2,
    'topic_3':topic_3,
    'topic_4':topic_4,
    'topic_5':topic_5,
    'topic_6':topic_6,
    'topic_7':topic_7,
    'topic_8':topic_8,
    'topic_9':topic_9,
    'topic_10':topic_10,
    'topic_11':topic_11,
    'topic_12':topic_12,
    'topic_13':topic_13,
    'topic_14':topic_14,
    'topic_15':topic_15,
    'topic_16':topic_16,
    'topic_17':topic_17,
    'topic_18':topic_18,
    'topic_19':topic_19,
    'topic_20':topic_20}
    

In [ ]:
topics_df = pd.DataFrame(list(df.apply(lambda x: get_topics(x.topic_dist), axis=1)))

In [ ]:
topics_df[['publicationName','field']] = df[['publicationName','field']].reset_index(drop=True)

In [ ]:
topics_summary = topics_df.groupby(['publicationName','field']).agg(mean)
topics_df.to_csv('../../results/topics_dist_by_article.csv', index=False)